In [7]:
cd ..

/Users/ali/Documents/CS tomfoolery/oleorcarl


In [10]:
from oleorcarl.database import db
from oleorcarl.database.models import Student

In [11]:
alistair = Student()

In [15]:
alistair.name = "Alistair Pattison"
alistair.email = "pattisona@carleton.edu"
vars(alistair)

{'__data__': {'name': 'Alistair Pattison', 'email': 'pattisona@carleton.edu'},
 '_dirty': {'email', 'name'},
 '__rel__': {},
 'test': 'cheeks'}

In [14]:
alistair.save()

0

In [26]:
Student.get_or_none() is None

True

In [16]:
from oleorcarl.scraper.spiders.carleton import _get_carleton_cookies

_get_carleton_cookies()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (carleton.py, line 51)